In [2]:
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from tqdm.auto import tqdm

def ir_srgan(img_path, save_path, netG):
    # Image saving path
    img_name = img_path[img_path.rfind("\\")+1:]
    lr_img = Image.open(img_path)
    #resize the image to 32x32
    pre_process_lr = transforms.Compose([transforms.ToTensor()])

    lr_img = pre_process_lr(lr_img)
    print("Shape of the input image: ",lr_img.shape)

    # show the image
    import matplotlib.pyplot as plt
    lr_img = lr_img.permute(1, 2, 0)
    print("Input image: ")
    plt.imshow(lr_img)
    plt.show()

    import torch
    with torch.no_grad():
        lr_img = lr_img.permute(2, 0, 1)
        sr_img = netG(lr_img.unsqueeze(0)) # c, h, w
        print("Shape of the Super Resolved image: ",sr_img.shape)
        sr_img = sr_img.squeeze(0).permute(1, 2, 0).clamp(0, 1).numpy()
        ing_file_name = os.path.join(save_path,img_name)
        sr_img = np.squeeze(sr_img)
        plt.imsave(ing_file_name, sr_img, cmap='gray')
        plt.imshow(sr_img)
        plt.show()

### Model Evaluation for IR Super resolution GAN 
* Pixel loss - L1
* Feature loss - L1
* No edge loss

In [1]:
%cd C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs
import torch
import torch.nn as nn
from RRDBNet_arch import RRDBNet

ir_edge_netG = RRDBNet(in_nc=1, out_nc=1, nf=64, nb=23, gc=32)

#load the otsr model
# model parameters when the psnr is around 21
path_0 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_L1\9494_G.pth'

# model parameters at the end of training at 40000th step
path_1 = r'C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs\trained_models_L1\40138_G.pth'
ir_edge_netG.load_state_dict(torch.load(path_0), strict=True)

C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_imgs


<All keys matched successfully>

#### Validation for FLIR Images 

In [ ]:
gt_img = r"C:\SaiVinay\Dataset\SRIR Data\Train"
img_list = os.listdir(gt_img)
img_path = [os.path.join(gt_img, x) for x in img_list]
for ixd in tqdm(range(len(img_path))):
    img_ = img_path[ixd]
    save_path = r"C:\SaiVinay\GitHub\ir_srgan_otsr\src_ir_with_edgeloss\val_imgs"
    ir_srgan(img_, save_path, ir_edge_netG)